# pandas.DataFrame 로드하기

이번 튜토리얼에서는 pandas 데이터프레임을 `tf.data.Dataset`으로 어떻게 로드하는지에 대해 학습해 보겠습니다.

이번에는 Cleveland Clinic Foundation for Heart Disease 에서 제공하는 작은 데이터세트를 사용합니다. CSV 안에는 수 백개의 행들이 있습니다. 각 행은 환자를 나타내고, 각 열은 환자의 상태를 나타냅니다. 우리는 이 정보들을 사용해서 환자가 심장병이 있는지의 여부를 예측하는 이진 분류 작업입니다.

## 1. pandas를 사용해 데이터 읽기

In [1]:
import pandas as pd
import tensorflow as tf

심장 데이터세트를 포함한 csv 파일을 다운로드 받습니다.

In [2]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


csv 파일을 pandas를 사용해 읽습니다.

In [3]:
df = pd.read_csv(csv_file)

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [5]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

데이터타입이 `object`인 `thal` 열을 수치화시킵니다.

In [6]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [7]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


## 2. `tf.data.Dataset`을 이용해 데이터 로드하기

pandas 데이터프레임으로부터 값을 읽기 위해 `tf.data.Dataset.from_tensor_slices`를 이용합니다.

`tf.data.Dataset`을 쓰는 강점 중에 하나는 간단하고 굉장히 효율적인 데이터 파이프라인을 작성하게 해준다는 것입니다.

In [8]:
target = df.pop('target')  # 심장병의 여부를 나타내는 레이블

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [10]:
for feat, targ in dataset.take(5):
    print('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


`pd.Series`는 `__array__` 프로토콜을 항상 포함하고 있어서 `np.array` 또는 `tf.Tensor`를 사용하는 거의 모든 곳에서 사용될 수 있습니다.

In [11]:
tf.constant(df['thal'])

<tf.Tensor: shape=(303,), dtype=int8, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3,
      

훈련 데이터세트를 셔플 및 배치합니다.

In [12]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## 3. 모델 생성 및 훈련

In [13]:
def get_compiled_model():
    model = tf.keras.Sequential([
                                 tf.keras.layers.Dense(10, activation='relu'),
                                 tf.keras.layers.Dense(10, activation='relu'),
                                 tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [14]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15
303/303 [==============================] - 5s 4ms/step - loss: 3.7504 - accuracy: 0.6898
Epoch 2/15
303/303 [==============================] - 1s 4ms/step - loss: 0.8359 - accuracy: 0.6898
Epoch 3/15
303/303 [==============================] - 1s 4ms/step - loss: 0.6899 - accuracy: 0.7129
Epoch 4/15
303/303 [==============================] - 1s 4ms/step - loss: 0.6147 - accuracy: 0.7030
Epoch 5/15
303/303 [==============================] - 1s 4ms/step - loss: 0.5742 - accuracy: 0.7228
Epoch 6/15
303/303 [==============================] - 1s 4ms/step - loss: 0.5442 - accuracy: 0.7327
Epoch 7/15
303/303 [==============================] - 1s 4ms/step - loss: 0.5447 - accuracy: 0.7294
Epoch 8/15
303/303 [==============================] - 1s 4ms/step - loss: 0.5259 - accuracy: 0.7459
Epoch 9/15
303/303 [==============================] - 1s 4ms/step - loss: 0.5227 - accuracy: 0.7360
Epoch 10/15
303/303 [==============================] - 1s 3ms/step - loss: 0.5008 - accuracy: 0.7525

딕셔너리를 input으로서 모델로 보내는 것은 `tf.keras.layers.Input` 레이어의 매칭 딕셔너리를 만드는 것으로 간단하게 구현할 수 있습니다. functional API를 사용해서 전처리와 스택 작업을 적용할 수 있습니다.

df의 열 대신 df를 딕셔너리화해서 모델을 구성하고 훈련시켜 보겠습니다.

In [15]:
# 모델 레이어 작업
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}  # 딕셔너리 화
x = tf.stack(list(inputs.values()), axis=-1)  # 딕셔너리의 value들을 모두 묶는다.

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

`tf.data`를 사용할 때 pandas 데이터프레임의 열 구조를 보존하는 가장 쉬운 방법은 `pd.DataFrame`을 `dict`로 변환해서 slice하는 것이다.

(앞서 slice할 때는 df.value로 행했습니다)

In [16]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [17]:
for dict_slice in dict_slices.take(1):
    print(dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57],
      dtype=int32)>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3], dtype=int32)>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150], dtype=int32)>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168], dtype=int32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int32)>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0], dtype=int32)>, 'thalach': <tf.Tensor: shape=(16

In [18]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 0s 3ms/step - loss: 118.5568 - accuracy: 0.2739
Epoch 2/15
19/19 [==============================] - 0s 3ms/step - loss: 102.1970 - accuracy: 0.2739
Epoch 3/15
19/19 [==============================] - 0s 3ms/step - loss: 87.2684 - accuracy: 0.2739
Epoch 4/15
19/19 [==============================] - 0s 3ms/step - loss: 71.1838 - accuracy: 0.2739
Epoch 5/15
19/19 [==============================] - 0s 3ms/step - loss: 53.8688 - accuracy: 0.2739
Epoch 6/15
19/19 [==============================] - 0s 3ms/step - loss: 36.4831 - accuracy: 0.2739
Epoch 7/15
19/19 [==============================] - 0s 3ms/step - loss: 19.9257 - accuracy: 0.2772
Epoch 8/15
19/19 [==============================] - 0s 3ms/step - loss: 6.6791 - accuracy: 0.4455
Epoch 9/15
19/19 [==============================] - 0s 3ms/step - loss: 2.8968 - accuracy: 0.6799
Epoch 10/15
19/19 [==============================] - 0s 3ms/step - loss: 2.6981 - accuracy: 0.7129
Epoch 11/1